In [12]:
# all the imports
from pathlib import Path
from datetime import datetime
from datetime import timedelta
import pandas as pd

In [13]:
curr_dir = Path.cwd()
data_dir = curr_dir / 'User_data'
log_file_path = []

# get the latest log file
for user in data_dir.iterdir():
    for session in user.iterdir():
        for log in session.iterdir():
            if log.suffix == '.log':
                log_file_path.append(log)


In [14]:
file_path = log_file_path[0]
print(file_path)

/home/niskal/Abhishek/OpenBCI_DataAnlaysis/User_data/12345/session_2/12345_session_2_22-03-2023_21-59.log


In [15]:
comm_list = []
sysmon_list = []
start_time = 0
end_time = 0

In [16]:
with open(file_path, 'r') as f:
    lines  = f.readlines()[2:]

start_time = lines[0].split()[0]
end_time = lines[-1].split()[0]
f.close()

In [17]:
# traverse thorugh the log file for sysmon and communication events
with open(file_path ,'r') as f:
    for line in f:
        line = line.split('\t')

        if(len(line)<=1):
            continue

        if line[2] == 'SYSMON':
            sysmon_list.append(line)

        if line[2] == 'COMMUN' and line[6]!='SELECTED\n':
            comm_list.append(line)

In [18]:
def create_dict(start_time, end_time):
    #make it in HH:MM:SS format
    start_time = start_time.split('.')[0]
    start =datetime.strptime(start_time, '%H:%M:%S')
    end_time = end_time.split('.')[0]
    end = datetime.strptime(end_time, '%H:%M:%S')
    time_diff = end - start
    time_list = [start + timedelta(seconds=x) for x in range(time_diff.seconds + 1)]
    time_dict = {time.strftime('%H:%M:%S'): 0 for time in time_list}
    return time_dict

In [19]:
#************************************COMMUNICATION MODULE************************************
def getCommScore(comm_list,time_dict):
    #first split on TARGET
    split_list = []
    response_times = []
    repsonse_accuracies = []
    i=0
    while i < len(comm_list):
        if comm_list[i][6] == 'TARGET':
            j = i+1
            while j < len(comm_list) and comm_list[j][6] != 'TARGET':
                j+=1
            split_list.append(comm_list[i:j])
            i = j
            continue
        i+=1

    #traverse through the split list and get the score
    for event in split_list:
        channel =''
        #case when the prompt is for the subject
        if event[0][4] == 'OWN':
            channel = event[0][5]
            curr_target = event[0][7].strip()
            #when ENTER is not pressed
            if event[-1][5] != 'RETURN':
                #add 30 seconds to the response time
                #add 0 to the accuracy
                response_times.append(30)
                repsonse_accuracies.append(0)

                #get the start time
                start_time = datetime.strptime(event[0][0], '%H:%M:%S.%f')
                start_time = start_time.strftime('%H:%M:%S')
                #get the end time
                end_time = datetime.strptime(event[-1][0], '%H:%M:%S.%f')
                end_time = end_time.strftime('%H:%M:%S')

                #update the time_dict
                for time in time_dict:
                    if time >= start_time and time <= end_time:
                        time_dict[time] = 1
                continue

            #when ENTER is pressed
            #get the last set frequency
            last_freq = event[-2][6].strip()
            if(curr_target == last_freq):
                #correct frequency is set
                repsonse_accuracies.append(1)
                start_time = datetime.strptime(event[0][0], '%H:%M:%S.%f')
                end_time = datetime.strptime(event[-2][0], '%H:%M:%S.%f')
                response_time = end_time - start_time
                response_times.append(response_time.total_seconds())
                continue

            #correct frequency is not set
            error = abs(float(curr_target) - float(last_freq))/float(curr_target)
            accuracy = 1 - error
            repsonse_accuracies.append(accuracy)
            start_time = datetime.strptime(event[0][0], '%H:%M:%S.%f')
            end_time = datetime.strptime(event[-2][0], '%H:%M:%S.%f')
            response_time = end_time - start_time
            response_times.append(response_time.total_seconds())
            start_time = start_time.strftime('%H:%M:%S')
            end_time = end_time.strftime('%H:%M:%S')
            #update the time_dicts
            for time in time_dict:
                if time >= start_time and time <= end_time:
                    time_dict[time] = 1


        #case when the prompt is not for the subject
        if event[0][4] == 'OTHER':
            channel = event[0][5]
            #check if the frequency is changed
            if(len(event)>=5):
                #frequency is changed
                response_times.append(30)
                repsonse_accuracies.append(0)
                #get the start time
                start_time = datetime.strptime(event[0][0], '%H:%M:%S.%f')
                start_time = start_time.strftime('%H:%M:%S')
                #get the end time
                end_time = datetime.strptime(event[-1][0], '%H:%M:%S.%f')
                end_time = end_time.strftime('%H:%M:%S')

                #update the time_dict
                for time in time_dict:
                    if time >= start_time and time <= end_time:
                        time_dict[time] = 1
            else:
                #frequency is not changed
                response_times.append(0)
                repsonse_accuracies.append(1)

    print(f'response times:{response_times}')
    print(f'response accuracies:{repsonse_accuracies}')
#********************************************************

In [20]:
time_dict  = create_dict(start_time, end_time)
getCommScore(comm_list, time_dict)

#print key hacing value 1
for key, value in time_dict.items():
    if value == 1:
        print(key)


response times:[15.213925, 0, 0, 30, 0, 16.173155, 0]
response accuracies:[1, 1, 1, 0, 1, 1, 1]
21:57:07
21:57:08
21:57:09
21:57:10
21:57:11
21:57:12
21:57:13
21:57:14
21:57:15
21:57:16
21:57:17
21:57:18
21:57:19
21:57:20
21:57:21
21:57:22
21:57:23


In [25]:
track_list = []
track_file = '/home/niskal/Abhishek/OpenBCI_DataAnlaysis/User_data/12345/session_2/12345_session_2_22-03-2023_21-59_track_log.txt'
with open(track_file ,'r') as tf:
    for line in tf:
        temp_line = line.split(' ')
        if(str(temp_line[0]) == 'OUT:'):
                time = temp_line[2].split('.')[0]
                print(time)
                time_dict[time] = 1
        
        track_list.append(line)

21:56:13
21:56:13
21:56:13
21:56:13
21:56:13
21:56:13
21:56:13
21:56:13
21:56:13
21:56:13
21:56:13
21:56:13
21:56:13
21:56:13
21:56:13
21:56:13
21:56:13
21:56:13
21:56:13
21:56:13
21:56:13
21:56:13
21:56:13
21:56:13
21:56:13
21:56:13
21:56:13
21:56:13
21:56:13
21:56:13
21:56:13
21:56:13
21:56:13
21:56:13
21:56:13
21:56:13
21:56:14
21:56:14
21:56:38
21:56:38
21:56:38
21:56:38
21:56:38
21:56:38
21:56:38
21:56:38
21:56:38
21:56:38
21:56:38
21:56:38
21:56:38
21:56:38
21:56:38
21:56:38
21:56:38
21:56:38
21:56:38
21:56:38
21:56:38
21:56:38
21:56:38
21:56:38
21:56:38
21:56:38
21:56:38
21:56:38
21:56:38
21:56:38
21:56:38
21:56:38
21:56:38
21:56:38
21:56:38
21:56:38
21:56:39
21:56:39
21:56:39
21:56:39
21:56:39
21:56:39
21:56:39
21:56:39
21:56:39
21:56:39
21:56:39
21:56:39
21:56:39
21:56:39
21:56:39
21:56:39
21:56:39
21:56:39
21:56:39
21:56:39
21:56:39
21:56:39
21:56:39
21:56:39
21:56:40
21:56:40
21:56:40
21:56:40
21:56:40
21:56:40
21:56:40
21:56:40
21:56:40
21:56:40
21:56:40
21:56:40
21:56:51
2